# Task 1

In [ ]:
import cv2
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import *
from moviepy import VideoFileClip
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression

def scale(x):
    x = abs(x)
    if x.mean() == 0:
        return np.array([])
    return x/x.mean()


df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))

ids = []
Driver_State_Changed = []
video_Driver_State_Changed = {}

chunk = 16
for video in tqdm(sorted(annotations.keys())):
    a = {}
    speed = {}
    traffic_scene_frames = {}
    traffic_scene_centroids = {}
    challenge_object_frames = {}
    challenge_object_centroids = {}
    num_frames = len(annotations[video].keys())
    for frame in sorted(annotations[video].keys()):
        for i in range(len(annotations[video][frame]['traffic_scene'])):
            if annotations[video][frame]['traffic_scene'][i]['track_id'] in traffic_scene_centroids:
                x1, y1, x2, y2 = annotations[video][frame]['traffic_scene'][i]['bbox']
                traffic_scene_frames[annotations[video][frame]['traffic_scene'][i]['track_id']].append(frame)
                traffic_scene_centroids[annotations[video][frame]['traffic_scene'][i]['track_id']].append([x1+(abs(x2-x1)/2),y1+(abs(y2-y1)/2)])
            else:
                x1, y1, x2, y2 = annotations[video][frame]['traffic_scene'][i]['bbox']
                traffic_scene_frames[annotations[video][frame]['traffic_scene'][i]['track_id']] = [frame]
                traffic_scene_centroids[annotations[video][frame]['traffic_scene'][i]['track_id']] = [[x1+(abs(x2-x1)/2),y1+(abs(y2-y1)/2)]]
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] in challenge_object_centroids:
                x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
                challenge_object_frames[annotations[video][frame]['challenge_object'][i]['track_id']].append(frame)
                challenge_object_centroids[annotations[video][frame]['challenge_object'][i]['track_id']].append([x1+(abs(x2-x1)/2),y1+(abs(y2-y1)/2)])
            else:
                x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
                challenge_object_frames[annotations[video][frame]['challenge_object'][i]['track_id']] = [frame]
                challenge_object_centroids[annotations[video][frame]['challenge_object'][i]['track_id']] = [[x1+(abs(x2-x1)/2),y1+(abs(y2-y1)/2)]]

        for k in traffic_scene_frames:
            if frame in traffic_scene_frames[k]:
                if len(traffic_scene_frames[k]) > 1:
                    y = np.linalg.norm(np.array(traffic_scene_centroids[k][1:])-np.array(traffic_scene_centroids[k][:-1]), axis=1)
                    x = np.array(traffic_scene_frames[k][1:]).reshape(-1, 1)-traffic_scene_frames[k][0]
                    speed_model = LinearRegression().fit(x, y)
                    if k in speed:
                        speed[k].append(speed_model.coef_)
                        if len(speed[k]) > chunk:
                            if k in a:
                                a[k].append([frame, float(LinearRegression().fit(np.arange(chunk).reshape(-1, 1), y[-chunk:]).coef_[0])])
                            else:
                                a[k] = [[frame, float(LinearRegression().fit(np.arange(chunk).reshape(-1, 1), y[-chunk:]).coef_[0])]]
                    else:
                        speed[k] = [speed_model.coef_]

        for k in challenge_object_frames:
            if frame in challenge_object_frames[k]:
                if len(challenge_object_frames[k]) > 1:
                    y = np.linalg.norm(np.array(challenge_object_centroids[k][1:])-np.array(challenge_object_centroids[k][:-1]), axis=1)
                    x = np.array(challenge_object_frames[k][1:]).reshape(-1, 1)-challenge_object_frames[k][0]
                    speed_model = LinearRegression().fit(x, y)
                    if k in speed:
                        speed[k].append(speed_model.coef_)
                        if len(speed[k]) > chunk:
                            if k in a:
                                a[k].append([frame, float(LinearRegression().fit(np.arange(chunk).reshape(-1, 1), y[-chunk:]).coef_[0])])
                            else:
                                a[k] = [[frame, float(LinearRegression().fit(np.arange(chunk).reshape(-1, 1), y[-chunk:]).coef_[0])]]
                    else:
                        speed[k] = [speed_model.coef_]
    speed_detect = {}
    for k, v in a.items():
        x = np.array(v)
        for d in x[:, 0][find_peaks(scale(x[:,1]),height=3.3, distance=40)[0]]:
            d = int(d)
            if d > chunk//2:
                if d in speed_detect:
                    speed_detect[d] += 1
                else:
                    speed_detect[d] = 1
                    
    sound_detect = {}
    clip = VideoFileClip(f"COOOL/{video}.mp4")
    signal = scale(np.abs(list(clip.audio.iter_frames(fps=60)))[:, 0])
    peaks, _ = find_peaks(signal, height=3.3, distance=40)
    for peak in peaks:
        d = round(num_frames*peak/len(signal))
        if d > chunk:
            if d in sound_detect:
                sound_detect[d] += 1
            else:
                sound_detect[d] = 1

    signal = scale(np.abs(list(clip.audio.iter_frames(fps=60)))[:, 1])
    peaks, _ = find_peaks(signal, height=3.3, distance=40)
    for peak in peaks:
        d = round(num_frames*peak/len(signal))
        if d > chunk:
            if d in sound_detect:
                sound_detect[d] += 1
            else:
                sound_detect[d] = 1

    signal = scale(np.mean(np.abs(list(clip.audio.iter_frames(fps=60))), axis=1))
    peaks, _ = find_peaks(signal, height=3.3, distance=40)
    for peak in peaks:
        d = round(num_frames*peak/len(signal))
        if d > chunk:
            if d in sound_detect:
                sound_detect[d] += 1
            else:
                sound_detect[d] = 1

    ids += [f"{video}_{frame}" for frame in range(num_frames)]
    if len(sound_detect) == 0 and len(speed_detect) == 0:
        Driver_State_Changed += [False]*int(0.25*num_frames) + [True]*(num_frames-int(0.25*num_frames))
        video_Driver_State_Changed[video] = ["random", int(0.25*num_frames)]
    elif len(sound_detect) == 0:
        Driver_State_Changed += [frame >= min(speed_detect) for frame in range(num_frames)]
        video_Driver_State_Changed[video] = ["Speed", min(speed_detect)]
    elif len(speed_detect) == 0:
        Driver_State_Changed += [frame >= np.average(list(sound_detect.keys()), weights=list(sound_detect.values())) for frame in range(num_frames)]
        video_Driver_State_Changed[video] = ["Sound", np.average(list(sound_detect.keys()), weights=list(sound_detect.values()))]
    else:
        Driver_State_Changed += [frame >= min(speed_detect) for frame in range(num_frames)]
        video_Driver_State_Changed[video] = ["Speed+Sound", min(speed_detect), np.average(list(sound_detect.keys()), weights=list(sound_detect.values()))]

with open('video_Driver_State_Changed.pkl', 'wb') as handle:
    pickle.dump(video_Driver_State_Changed, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Task 2

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import *
from moviepy import VideoFileClip
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression
from transformers import AutoImageProcessor, AutoModelForImageClassification

def scale(x):
    x = abs(x)
    if x.mean() == 0:
        return np.array([])
    return x/x.mean()

def is_point_in_quadrilateral(x, y, points):
    def cross_product(x1, y1, x2, y2, x, y):
        return (x2 - x1) * (y - y1) - (y2 - y1) * (x - x1)
    
    P1, P2, P3, P4 = points  # bottom_left, top_left, bottom_right, top_right
    C1 = cross_product(P1[0], P1[1], P2[0], P2[1], x, y)
    C2 = cross_product(P2[0], P2[1], P4[0], P4[1], x, y)
    C3 = cross_product(P4[0], P4[1], P3[0], P3[1], x, y)
    C4 = cross_product(P3[0], P3[1], P1[0], P1[1], x, y)

    # Check if all cross products have the same sign
    return (C1 >= 0 and C2 >= 0 and C3 >= 0 and C4 >= 0) or (C1 <= 0 and C2 <= 0 and C3 <= 0 and C4 <= 0)


def imagenet_map(x):
    if x in [407, 408, 468, 569, 609, 627, 654, 656, 675, 734, 817, 867, 874]:
        return "car"
    return model.config.id2label[x]

ids = []
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window16-256")
model = AutoModelForImageClassification.from_pretrained("microsoft/swinv2-base-patch4-window16-256").to("cuda", dtype=torch.float16)

processor2 = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model2 = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224").to("cuda", dtype=torch.float16)

Hazard_Track_all = [[] for i in range(23)]
video_track_id = {}
video_first_hazard = {}
weight = np.array([ 0.05, 1.0, 0.04, 0.5, 1.0,  0.04 , 0.67, -0.9])

for video in tqdm(sorted(list(annotations.keys()))):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    challenge_object_frames = {}
    challenge_object_labels = {}
    challenge_object_labels2 = {}
    challenge_object_centroids = {}
    challenge_object_minx = {}
    challenge_object_maxx = {}
    num_frames = len(annotations[video].keys())
    
    all_centroids = {}
    count_frame = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)
        rows, cols = frame_image.shape[:2]
        bottom_left  = [int(cols * 0.05), int(rows * 0.8)]
        top_left     = [int(cols * 0.48), int(rows * 0.42)]
        bottom_right = [int(cols * 0.95), int(rows * 0.95)]
        top_right    = [int(cols * 0.55), int(rows * 0.4)]

        chips = []
        track_ids = []
        all_centroids[frame] = {}
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] not in challenge_object_centroids:
                count_frame[annotations[video][frame]['challenge_object'][i]['track_id']] = 0
                challenge_object_frames[annotations[video][frame]['challenge_object'][i]['track_id']] = []
                challenge_object_centroids[annotations[video][frame]['challenge_object'][i]['track_id']] = []
                challenge_object_minx[annotations[video][frame]['challenge_object'][i]['track_id']] = 999999999999
                challenge_object_maxx[annotations[video][frame]['challenge_object'][i]['track_id']] = -1000000
                
            x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
            challenge_object_frames[annotations[video][frame]['challenge_object'][i]['track_id']].append(frame)
            challenge_object_centroids[annotations[video][frame]['challenge_object'][i]['track_id']].append([x1+(abs(x2-x1)/2),y1+(abs(y2-y1)/2)])
            challenge_object_minx[annotations[video][frame]['challenge_object'][i]['track_id']] = min(challenge_object_minx[annotations[video][frame]['challenge_object'][i]['track_id']], x1, x2)
            challenge_object_maxx[annotations[video][frame]['challenge_object'][i]['track_id']] = max(challenge_object_maxx[annotations[video][frame]['challenge_object'][i]['track_id']], x1, x2)
            chips.append( frame_image[max(0, int(1.2*y1-0.2*y2)):min(int(1.2*y2-0.2*y1), frame_image.shape[0]), max(0, int(1.2*x1-0.2*x2)):min(int(1.2*x2-0.2*x1), frame_image.shape[1])] )
            track_ids.append(annotations[video][frame]['challenge_object'][i]['track_id'])
            all_centroids[frame][annotations[video][frame]['challenge_object'][i]['track_id']] = [x1+(abs(x2-x1)/2),y1+(abs(y2-y1)/2)]
            count_frame[annotations[video][frame]['challenge_object'][i]['track_id']] += 1
            
            if annotations[video][frame]['challenge_object'][i]['track_id'] not in challenge_object_labels:
                challenge_object_labels[annotations[video][frame]['challenge_object'][i]['track_id']] = {}
                challenge_object_labels2[annotations[video][frame]['challenge_object'][i]['track_id']] = {}

        if len(chips) == 0:
            continue
        inputs = processor(images=chips, return_tensors="pt").to("cuda", dtype=torch.float16)
        predicted_class_idx = model(**inputs).logits.argmax(-1).detach().cpu().numpy().tolist()
        for track_id, label, image in zip(track_ids, predicted_class_idx, chips):
            label = imagenet_map(label)
            if label not in challenge_object_labels[track_id]:
                challenge_object_labels[track_id][label] = 0.0
            challenge_object_labels[track_id][label] += image.shape[0]*image.shape[1]

        inputs = processor2(images=chips, return_tensors="pt").to("cuda", dtype=torch.float16)
        predicted_class_idx = model2(**inputs).logits.argmax(-1).detach().cpu().numpy().tolist()
        for track_id, label, image in zip(track_ids, predicted_class_idx, chips):
            label = imagenet_map(label)
            if label not in challenge_object_labels2[track_id]:
                challenge_object_labels2[track_id][label] = 0.0
            challenge_object_labels2[track_id][label] += image.shape[0]*image.shape[1]

    hazards_prob = {k: [1.0, 0.0, 9999999, 0, 0, 1.0, 1.0, 1.0] for k in challenge_object_labels.keys()}
    # 1 car and center
    for k in challenge_object_labels.keys():
        if (max(challenge_object_labels[k].keys(), key=challenge_object_labels[k].get) == "car") and (challenge_object_labels[k]["car"]/sum(challenge_object_labels[k].values()) > 0.3):
            hazards_prob[k][0] = -10.0
        else:
            if "car" in challenge_object_labels[k]:
                hazards_prob[k][0] = hazards_prob[k][0] - challenge_object_labels[k]["car"]/sum(challenge_object_labels[k].values())

    for frame in all_centroids.keys():
        track_ids = []
        centroids = []
        for k in all_centroids[frame].keys():
            if hazards_prob[k][0] > 0.2:
                track_ids.append(k)
                centroids.append(all_centroids[frame][k])
    
        if len(centroids) > 0:
            image_center = [frame_image.shape[1]/2, frame_image.shape[0]]
            potential_hazard_dists = np.linalg.norm(np.array(centroids)-image_center, axis=1)
            probable_hazard = np.argmin(potential_hazard_dists)
            hazards_prob[track_ids[probable_hazard]][2] = min(float(hazards_prob[track_ids[probable_hazard]][2]), potential_hazard_dists[probable_hazard])

    for k, v in hazards_prob.items():
        hazards_prob[k][2] = 1.0-float(hazards_prob[k][2]/max(x[2] for x in hazards_prob.values() if x != 9999999))

    # 2 left-right
    for k in challenge_object_frames:
        hazards_prob[k][1] = (challenge_object_maxx[k] - challenge_object_minx[k])/frame_image.shape[0] - 0.2*abs(challenge_object_centroids[k][0][0] - challenge_object_centroids[k][-1][0])/frame_image.shape[0]
    
    for k, v in hazards_prob.items():
        hazards_prob[k][3] = float(np.mean([is_point_in_quadrilateral(x, y, [bottom_left, top_left, bottom_right, top_right]) for x, y in challenge_object_centroids[k]])) - 0.04*float(any([is_point_in_quadrilateral(x, y, [bottom_left, top_left, bottom_right, top_right]) for x, y in challenge_object_centroids[k]]))

    for k in challenge_object_frames:
        hazards_prob[k][4] = 1/len(challenge_object_frames.keys())

    for k in challenge_object_labels2.keys():
        if (max(challenge_object_labels2[k].keys(), key=challenge_object_labels2[k].get) == "car") and (challenge_object_labels2[k]["car"]/sum(challenge_object_labels2[k].values()) > 0.3):
            hazards_prob[k][5] = -10.0
        else:
            if "car" in challenge_object_labels2[k]:
                hazards_prob[k][5] = hazards_prob[k][5] - challenge_object_labels2[k]["car"]/sum(challenge_object_labels2[k].values())

    for k in count_frame:
        hazards_prob[k][6] = 1- count_frame[k]/num_frames
    
    video_track_id[video] = []
    result = {}
    for k, v in hazards_prob.items():
        result[k] = sum( weight*np.random.normal(loc=1.0, scale=0.01, size=8)*np.array(v) )  # apply dp -> ensemble

    video_track_id[video] += [k for i, k in enumerate(sorted(result, key=result.get, reverse = True)) if (result[k] > 0  or i == 0)]
    Hazard_Track_video = [[-1]*num_frames for i in range(23)]
    for frame in range(num_frames):
        c = 0
        for track_id in video_track_id[video]:
            if track_id in [x['track_id'] for x in annotations[video][frame]['challenge_object']]:
                Hazard_Track_video[c][frame] = track_id
                if video not in video_first_hazard:
                    video_first_hazard[video] = frame
                c += 1
            if c == 23:
                print("out")
                break
        
    for i in range(23):
        Hazard_Track_all[i] += Hazard_Track_video[i]
    print(f"'{video}': {video_track_id[video]}")
    ids += [f"{video}_{frame}" for frame in range(num_frames)]

with open('video_track_id_1.pkl', 'wb') as handle:
    pickle.dump(video_track_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm.notebook import *
from moviepy import VideoFileClip
from scipy.signal import find_peaks
from sklearn.linear_model import LinearRegression
from transformers import AutoImageProcessor, AutoModelForImageClassification

def scale(x):
    x = abs(x)
    if x.mean() == 0:
        return np.array([])
    return x/x.mean()

def is_point_in_quadrilateral(x, y, points):
    def cross_product(x1, y1, x2, y2, x, y):
        return (x2 - x1) * (y - y1) - (y2 - y1) * (x - x1)
    
    P1, P2, P3, P4 = points  # bottom_left, top_left, bottom_right, top_right
    C1 = cross_product(P1[0], P1[1], P2[0], P2[1], x, y)
    C2 = cross_product(P2[0], P2[1], P4[0], P4[1], x, y)
    C3 = cross_product(P4[0], P4[1], P3[0], P3[1], x, y)
    C4 = cross_product(P3[0], P3[1], P1[0], P1[1], x, y)

    # Check if all cross products have the same sign
    return (C1 >= 0 and C2 >= 0 and C3 >= 0 and C4 >= 0) or (C1 <= 0 and C2 <= 0 and C3 <= 0 and C4 <= 0)


def imagenet_map(x):
    if x in [407, 408, 468, 569, 609, 627, 654, 656, 675, 734, 817, 867, 874]:
        return "car"
    return model.config.id2label[x]

ids = []
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
processor = AutoImageProcessor.from_pretrained("microsoft/swinv2-base-patch4-window16-256")
model = AutoModelForImageClassification.from_pretrained("microsoft/swinv2-base-patch4-window16-256").to("cuda", dtype=torch.float16)

processor2 = AutoImageProcessor.from_pretrained("google/vit-base-patch16-224")
model2 = AutoModelForImageClassification.from_pretrained("google/vit-base-patch16-224").to("cuda", dtype=torch.float16)

Hazard_Track_all = [[] for i in range(23)]
video_track_id = {}
video_first_hazard = {}
weight = np.array([0.054, 1.0, 0.0012, 0.838, 0.935, 0.0484, 0.707818, -1.3])

for video in tqdm(sorted(list(annotations.keys()))):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    challenge_object_frames = {}
    challenge_object_labels = {}
    challenge_object_labels2 = {}
    challenge_object_centroids = {}
    challenge_object_minx = {}
    challenge_object_maxx = {}
    num_frames = len(annotations[video].keys())
    
    all_centroids = {}
    count_frame = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)
        rows, cols = frame_image.shape[:2]
        bottom_left  = [int(cols * 0.05), int(rows * 0.8)]
        top_left     = [int(cols * 0.48), int(rows * 0.42)]
        bottom_right = [int(cols * 0.95), int(rows * 0.95)]
        top_right    = [int(cols * 0.55), int(rows * 0.4)]

        chips = []
        track_ids = []
        all_centroids[frame] = {}
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] not in challenge_object_centroids:
                count_frame[annotations[video][frame]['challenge_object'][i]['track_id']] = 0
                challenge_object_frames[annotations[video][frame]['challenge_object'][i]['track_id']] = []
                challenge_object_centroids[annotations[video][frame]['challenge_object'][i]['track_id']] = []
                challenge_object_minx[annotations[video][frame]['challenge_object'][i]['track_id']] = 999999999999
                challenge_object_maxx[annotations[video][frame]['challenge_object'][i]['track_id']] = -1000000
                
            x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
            challenge_object_frames[annotations[video][frame]['challenge_object'][i]['track_id']].append(frame)
            challenge_object_centroids[annotations[video][frame]['challenge_object'][i]['track_id']].append([x1+(abs(x2-x1)/2),y1+(abs(y2-y1)/2)])
            challenge_object_minx[annotations[video][frame]['challenge_object'][i]['track_id']] = min(challenge_object_minx[annotations[video][frame]['challenge_object'][i]['track_id']], x1, x2)
            challenge_object_maxx[annotations[video][frame]['challenge_object'][i]['track_id']] = max(challenge_object_maxx[annotations[video][frame]['challenge_object'][i]['track_id']], x1, x2)
            chips.append( frame_image[max(0, int(1.2*y1-0.2*y2)):min(int(1.2*y2-0.2*y1), frame_image.shape[0]), max(0, int(1.2*x1-0.2*x2)):min(int(1.2*x2-0.2*x1), frame_image.shape[1])] )
            track_ids.append(annotations[video][frame]['challenge_object'][i]['track_id'])
            all_centroids[frame][annotations[video][frame]['challenge_object'][i]['track_id']] = [x1+(abs(x2-x1)/2),y1+(abs(y2-y1)/2)]
            count_frame[annotations[video][frame]['challenge_object'][i]['track_id']] += 1
            
            if annotations[video][frame]['challenge_object'][i]['track_id'] not in challenge_object_labels:
                challenge_object_labels[annotations[video][frame]['challenge_object'][i]['track_id']] = {}
                challenge_object_labels2[annotations[video][frame]['challenge_object'][i]['track_id']] = {}

        if len(chips) == 0:
            continue
        inputs = processor(images=chips, return_tensors="pt").to("cuda", dtype=torch.float16)
        predicted_class_idx = model(**inputs).logits.argmax(-1).detach().cpu().numpy().tolist()
        for track_id, label, image in zip(track_ids, predicted_class_idx, chips):
            label = imagenet_map(label)
            if label not in challenge_object_labels[track_id]:
                challenge_object_labels[track_id][label] = 0.0
            challenge_object_labels[track_id][label] += image.shape[0]*image.shape[1]

        inputs = processor2(images=chips, return_tensors="pt").to("cuda", dtype=torch.float16)
        predicted_class_idx = model2(**inputs).logits.argmax(-1).detach().cpu().numpy().tolist()
        for track_id, label, image in zip(track_ids, predicted_class_idx, chips):
            label = imagenet_map(label)
            if label not in challenge_object_labels2[track_id]:
                challenge_object_labels2[track_id][label] = 0.0
            challenge_object_labels2[track_id][label] += image.shape[0]*image.shape[1]

    hazards_prob = {k: [1.0, 0.0, 9999999, 0, 0, 1.0, 1.0, 1.0] for k in challenge_object_labels.keys()}
    # 1 car and center
    for k in challenge_object_labels.keys():
        if (max(challenge_object_labels[k].keys(), key=challenge_object_labels[k].get) == "car") and (challenge_object_labels[k]["car"]/sum(challenge_object_labels[k].values()) > 0.3):
            hazards_prob[k][0] = -10.0
        else:
            if "car" in challenge_object_labels[k]:
                hazards_prob[k][0] = hazards_prob[k][0] - challenge_object_labels[k]["car"]/sum(challenge_object_labels[k].values())

    for frame in all_centroids.keys():
        track_ids = []
        centroids = []
        for k in all_centroids[frame].keys():
            if hazards_prob[k][0] > 0.2:
                track_ids.append(k)
                centroids.append(all_centroids[frame][k])
    
        if len(centroids) > 0:
            image_center = [frame_image.shape[1]/2, frame_image.shape[0]]
            potential_hazard_dists = np.linalg.norm(np.array(centroids)-image_center, axis=1)
            probable_hazard = np.argmin(potential_hazard_dists)
            hazards_prob[track_ids[probable_hazard]][2] = min(float(hazards_prob[track_ids[probable_hazard]][2]), potential_hazard_dists[probable_hazard])

    for k, v in hazards_prob.items():
        hazards_prob[k][2] = 1.0-float(hazards_prob[k][2]/max(x[2] for x in hazards_prob.values() if x != 9999999))

    # 2 left-right
    for k in challenge_object_frames:
        hazards_prob[k][1] = (challenge_object_maxx[k] - challenge_object_minx[k])/frame_image.shape[0] - 0.15*abs(challenge_object_centroids[k][0][0] - challenge_object_centroids[k][-1][0])/frame_image.shape[0]
    
    for k, v in hazards_prob.items():
        hazards_prob[k][3] = float(np.mean([is_point_in_quadrilateral(x, y, [bottom_left, top_left, bottom_right, top_right]) for x, y in challenge_object_centroids[k]])) -0.12*float(any([is_point_in_quadrilateral(x, y, [bottom_left, top_left, bottom_right, top_right]) for x, y in challenge_object_centroids[k]]))

    for k in challenge_object_frames:
        hazards_prob[k][4] = 1/len(challenge_object_frames.keys())

    for k in challenge_object_labels2.keys():
        if (max(challenge_object_labels2[k].keys(), key=challenge_object_labels2[k].get) == "car") and (challenge_object_labels2[k]["car"]/sum(challenge_object_labels2[k].values()) > 0.3):
            hazards_prob[k][5] = -10.0
        else:
            if "car" in challenge_object_labels2[k]:
                hazards_prob[k][5] = hazards_prob[k][5] - challenge_object_labels2[k]["car"]/sum(challenge_object_labels2[k].values())

    for k in count_frame:
        hazards_prob[k][6] = 1- count_frame[k]/num_frames
    
    video_track_id[video] = []
    result = {}
    for k, v in hazards_prob.items():
        result[k] = sum( weight*np.random.normal(loc=1.0, scale=0.01, size=8)*np.array(v) ) # apply dp -> ensemble

    video_track_id[video] += [k for i, k in enumerate(sorted(result, key=result.get, reverse = True)) if (result[k] > 0  or i == 0)]
    Hazard_Track_video = [[-1]*num_frames for i in range(23)]
    for frame in range(num_frames):
        c = 0
        for track_id in video_track_id[video]:
            if track_id in [x['track_id'] for x in annotations[video][frame]['challenge_object']]:
                Hazard_Track_video[c][frame] = track_id
                if video not in video_first_hazard:
                    video_first_hazard[video] = frame
                c += 1
            if c == 23:
                print("out")
                break
        
    for i in range(23):
        Hazard_Track_all[i] += Hazard_Track_video[i]
    print(f"'{video}': {video_track_id[video]}")
    ids += [f"{video}_{frame}" for frame in range(num_frames)]

with open('video_track_id_2.pkl', 'wb') as handle:
    pickle.dump(video_track_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Task 3

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from tqdm.notebook import *
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BlipProcessor, BlipForConditionalGeneration, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from clip_interrogator import Config, Interrogator

batchsize = 8
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
video_track_id = pickle.load(open("video_track_id.pkl", 'rb'))
video_track_id_tree = pickle.load(open("video_track_id_tree.pkl", 'rb'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16, device_map="cuda")

# processor = BlipProcessor.from_pretrained("noamrot/FuseCap")
# model= BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-6.7b")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")
# model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-itm-vit-g")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-itm-vit-g", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")


hazard_name_by_id = {} # pickle.load(open("hazard_name_by_id_final.pkl", "rb"))
def clean_text(text):
    text = text.replace("car view of ", "").replace(",", "").split()
    i = 1
    while i !=  len(text):
        if (text[i] == text[i-1]) or (text[i] == ""):
            text.pop(i)
        else:
            i += 1
    text[0] = text[0][0].upper() + text[0][1:]
    return " ".join(text)

for video in tqdm(sorted(annotations.keys())):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    num_frames = len(annotations[video].keys())
    batch_id = []
    batch_image = []
    if video not in hazard_name_by_id:
        hazard_name_by_id[video] = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)
        
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] in video_track_id[video] + video_track_id_tree[video] :
                x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
                batch_id.append(annotations[video][frame]['challenge_object'][i]['track_id'])
                batch_image.append( frame_image[max(0, int(1.1*y1-0.1*y2)):min(int(1.1*y2-0.1*y1), frame_image.shape[0]), max(0, int(1.1*x1-0.1*x2)):min(int(1.1*x2-0.1*x1), frame_image.shape[1])] )
                if annotations[video][frame]['challenge_object'][i]['track_id'] not in hazard_name_by_id[video]:
                    hazard_name_by_id[video][annotations[video][frame]['challenge_object'][i]['track_id']] = {}
                
        if ((len(batch_id) >= batchsize) or (frame == num_frames-1)) and len(batch_id) > 0:
            inputs = processor(batch_image, ["car view of"]*len(batch_image), return_tensors="pt").to(device, torch.float16)
            output = model.generate(**inputs, max_length=64)
            output_text = [clean_text(processor.decode(output[i], skip_special_tokens=True)) for i in range(len(batch_id))]

            for track_id, image, text in zip(batch_id, batch_image, output_text):
                if text not in hazard_name_by_id[video][track_id]:
                    hazard_name_by_id[video][track_id][text] = 0.0
                hazard_name_by_id[video][track_id][text] += image.shape[0]*image.shape[1]
            
            batch_id = []
            batch_image = []

with open('hazard_name_by_id_blip2flan.pkl', 'wb') as handle:
    pickle.dump(hazard_name_by_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from tqdm.notebook import *
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BlipProcessor, BlipForConditionalGeneration, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from clip_interrogator import Config, Interrogator

batchsize = 8
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
video_track_id = pickle.load(open("video_track_id.pkl", 'rb'))
video_track_id_tree = pickle.load(open("video_track_id_tree.pkl", 'rb'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16, device_map="cuda")

# processor = BlipProcessor.from_pretrained("noamrot/FuseCap")
# model= BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap", torch_dtype=torch.float16, device_map="cuda")

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-6.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")
# model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-itm-vit-g")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-itm-vit-g", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")


hazard_name_by_id = {} # pickle.load(open("hazard_name_by_id_final.pkl", "rb"))
def clean_text(text):
    text = text.replace("car view of ", "").replace(",", "").split()
    i = 1
    while i !=  len(text):
        if (text[i] == text[i-1]) or (text[i] == ""):
            text.pop(i)
        else:
            i += 1
    text[0] = text[0][0].upper() + text[0][1:]
    return " ".join(text)

for video in tqdm(sorted(annotations.keys())):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    num_frames = len(annotations[video].keys())
    batch_id = []
    batch_image = []
    if video not in hazard_name_by_id:
        hazard_name_by_id[video] = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)
        
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] in video_track_id[video] + video_track_id_tree[video] :
                x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
                batch_id.append(annotations[video][frame]['challenge_object'][i]['track_id'])
                batch_image.append( frame_image[max(0, int(1.1*y1-0.1*y2)):min(int(1.1*y2-0.1*y1), frame_image.shape[0]), max(0, int(1.1*x1-0.1*x2)):min(int(1.1*x2-0.1*x1), frame_image.shape[1])] )
                if annotations[video][frame]['challenge_object'][i]['track_id'] not in hazard_name_by_id[video]:
                    hazard_name_by_id[video][annotations[video][frame]['challenge_object'][i]['track_id']] = {}
                
        if ((len(batch_id) >= batchsize) or (frame == num_frames-1)) and len(batch_id) > 0:
            inputs = processor(batch_image, ["car view of"]*len(batch_image), return_tensors="pt").to(device, torch.float16)
            output = model.generate(**inputs, max_length=64)
            output_text = [clean_text(processor.decode(output[i], skip_special_tokens=True)) for i in range(len(batch_id))]

            for track_id, image, text in zip(batch_id, batch_image, output_text):
                if text not in hazard_name_by_id[video][track_id]:
                    hazard_name_by_id[video][track_id][text] = 0.0
                hazard_name_by_id[video][track_id][text] += image.shape[0]*image.shape[1]
            
            batch_id = []
            batch_image = []

with open('hazard_name_by_id_blip2opt6_4b.pkl', 'wb') as handle:
    pickle.dump(hazard_name_by_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from tqdm.notebook import *
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BlipProcessor, BlipForConditionalGeneration, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from clip_interrogator import Config, Interrogator

batchsize = 8
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
video_track_id = pickle.load(open("video_track_id.pkl", 'rb'))
video_track_id_tree = pickle.load(open("video_track_id_tree.pkl", 'rb'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16, device_map="cuda")

# processor = BlipProcessor.from_pretrained("noamrot/FuseCap")
# model= BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-6.7b")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")
# model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-itm-vit-g")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-itm-vit-g", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")


hazard_name_by_id = {} # pickle.load(open("hazard_name_by_id_final.pkl", "rb"))
def clean_text(text):
    text = text.replace("car view of ", "").replace(",", "").split()
    i = 1
    while i !=  len(text):
        if (text[i] == text[i-1]) or (text[i] == ""):
            text.pop(i)
        else:
            i += 1
    text[0] = text[0][0].upper() + text[0][1:]
    return " ".join(text)

for video in tqdm(sorted(annotations.keys())):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    num_frames = len(annotations[video].keys())
    batch_id = []
    batch_image = []
    if video not in hazard_name_by_id:
        hazard_name_by_id[video] = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)
        
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] in video_track_id[video] + video_track_id_tree[video] :
                x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
                batch_id.append(annotations[video][frame]['challenge_object'][i]['track_id'])
                batch_image.append( frame_image[max(0, int(1.1*y1-0.1*y2)):min(int(1.1*y2-0.1*y1), frame_image.shape[0]), max(0, int(1.1*x1-0.1*x2)):min(int(1.1*x2-0.1*x1), frame_image.shape[1])] )
                if annotations[video][frame]['challenge_object'][i]['track_id'] not in hazard_name_by_id[video]:
                    hazard_name_by_id[video][annotations[video][frame]['challenge_object'][i]['track_id']] = {}
                
        if ((len(batch_id) >= batchsize) or (frame == num_frames-1)) and len(batch_id) > 0:
            inputs = processor(batch_image, ["car view of"]*len(batch_image), return_tensors="pt").to(device, torch.float16)
            output = model.generate(**inputs, max_length=64)
            output_text = [clean_text(processor.decode(output[i], skip_special_tokens=True)) for i in range(len(batch_id))]

            for track_id, image, text in zip(batch_id, batch_image, output_text):
                if text not in hazard_name_by_id[video][track_id]:
                    hazard_name_by_id[video][track_id][text] = 0.0
                hazard_name_by_id[video][track_id][text] += image.shape[0]*image.shape[1]
            
            batch_id = []
            batch_image = []

with open('hazard_name_by_id_blip.pkl', 'wb') as handle:
    pickle.dump(hazard_name_by_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from tqdm.notebook import *
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BlipProcessor, BlipForConditionalGeneration, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from clip_interrogator import Config, Interrogator

batchsize = 8
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
video_track_id = pickle.load(open("video_track_id.pkl", 'rb'))
video_track_id_tree = pickle.load(open("video_track_id_tree.pkl", 'rb'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16, device_map="cuda")

# processor = BlipProcessor.from_pretrained("noamrot/FuseCap")
# model= BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-6.7b")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")
# model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-itm-vit-g")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-itm-vit-g", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")


hazard_name_by_id = {} # pickle.load(open("hazard_name_by_id_final.pkl", "rb"))
def clean_text(text):
    text = text.replace("car view of ", "").replace(",", "").split()
    i = 1
    while i !=  len(text):
        if (text[i] == text[i-1]) or (text[i] == ""):
            text.pop(i)
        else:
            i += 1
    text[0] = text[0][0].upper() + text[0][1:]
    return " ".join(text)

for video in tqdm(sorted(annotations.keys())):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    num_frames = len(annotations[video].keys())
    batch_id = []
    batch_image = []
    if video not in hazard_name_by_id:
        hazard_name_by_id[video] = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)
        
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] in video_track_id[video] + video_track_id_tree[video] :
                x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
                batch_id.append(annotations[video][frame]['challenge_object'][i]['track_id'])
                batch_image.append( frame_image[max(0, int(1.1*y1-0.1*y2)):min(int(1.1*y2-0.1*y1), frame_image.shape[0]), max(0, int(1.1*x1-0.1*x2)):min(int(1.1*x2-0.1*x1), frame_image.shape[1])] )
                if annotations[video][frame]['challenge_object'][i]['track_id'] not in hazard_name_by_id[video]:
                    hazard_name_by_id[video][annotations[video][frame]['challenge_object'][i]['track_id']] = {}
                
        if ((len(batch_id) >= batchsize) or (frame == num_frames-1)) and len(batch_id) > 0:
            inputs = processor(batch_image, ["car view of"]*len(batch_image), return_tensors="pt").to(device, torch.float16)
            output = model.generate(**inputs, max_length=64)
            output_text = [clean_text(processor.decode(output[i], skip_special_tokens=True)) for i in range(len(batch_id))]

            for track_id, image, text in zip(batch_id, batch_image, output_text):
                if text not in hazard_name_by_id[video][track_id]:
                    hazard_name_by_id[video][track_id][text] = 0.0
                hazard_name_by_id[video][track_id][text] += image.shape[0]*image.shape[1]
            
            batch_id = []
            batch_image = []

with open('hazard_name_by_id_blip2flan_4b.pkl', 'wb') as handle:
    pickle.dump(hazard_name_by_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from tqdm.notebook import *
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BlipProcessor, BlipForConditionalGeneration, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from clip_interrogator import Config, Interrogator

batchsize = 8
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
video_track_id = pickle.load(open("video_track_id.pkl", 'rb'))
video_track_id_tree = pickle.load(open("video_track_id_tree.pkl", 'rb'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16, device_map="cuda")

# processor = BlipProcessor.from_pretrained("noamrot/FuseCap")
# model= BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-6.7b")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

processor = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")
model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-itm-vit-g")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-itm-vit-g", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")


hazard_name_by_id = {} # pickle.load(open("hazard_name_by_id_final.pkl", "rb"))
def clean_text(text):
    text = text.replace("car view of ", "").replace(",", "").split()
    i = 1
    while i !=  len(text):
        if (text[i] == text[i-1]) or (text[i] == ""):
            text.pop(i)
        else:
            i += 1
    text[0] = text[0][0].upper() + text[0][1:]
    return " ".join(text)

for video in tqdm(sorted(annotations.keys())):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    num_frames = len(annotations[video].keys())
    batch_id = []
    batch_image = []
    if video not in hazard_name_by_id:
        hazard_name_by_id[video] = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)
        
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] in video_track_id[video] + video_track_id_tree[video] :
                x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
                batch_id.append(annotations[video][frame]['challenge_object'][i]['track_id'])
                batch_image.append( frame_image[max(0, int(1.1*y1-0.1*y2)):min(int(1.1*y2-0.1*y1), frame_image.shape[0]), max(0, int(1.1*x1-0.1*x2)):min(int(1.1*x2-0.1*x1), frame_image.shape[1])] )
                if annotations[video][frame]['challenge_object'][i]['track_id'] not in hazard_name_by_id[video]:
                    hazard_name_by_id[video][annotations[video][frame]['challenge_object'][i]['track_id']] = {}
                
        if ((len(batch_id) >= batchsize) or (frame == num_frames-1)) and len(batch_id) > 0:
            inputs = feature_extractor(batch_image, return_tensors="pt").to(device, torch.float16)
            output = model.generate(**inputs, max_length=64)
            output_text = [clean_text(processor.decode(output[i], skip_special_tokens=True)) for i in range(len(batch_id))]

            for track_id, image, text in zip(batch_id, batch_image, output_text):
                if text not in hazard_name_by_id[video][track_id]:
                    hazard_name_by_id[video][track_id][text] = 0.0
                hazard_name_by_id[video][track_id][text] += image.shape[0]*image.shape[1]
            
            batch_id = []
            batch_image = []

with open('hazard_name_by_id_vit.pkl', 'wb') as handle:
    pickle.dump(hazard_name_by_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from tqdm.notebook import *
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BlipProcessor, BlipForConditionalGeneration, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer
from clip_interrogator import Config, Interrogator

batchsize = 8
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
video_track_id = pickle.load(open("video_track_id.pkl", 'rb'))
video_track_id_tree = pickle.load(open("video_track_id_tree.pkl", 'rb'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16, device_map="cuda")

# processor = BlipProcessor.from_pretrained("noamrot/FuseCap")
# model= BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap", torch_dtype=torch.float16, device_map="cuda")

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-6.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")
# model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-itm-vit-g")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-itm-vit-g", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")


hazard_name_by_id = {} # pickle.load(open("hazard_name_by_id_final.pkl", "rb"))
def clean_text(text):
    text = text.replace("car view of ", "").replace(",", "").split()
    i = 1
    while i !=  len(text):
        if (text[i] == text[i-1]) or (text[i] == ""):
            text.pop(i)
        else:
            i += 1
    text[0] = text[0][0].upper() + text[0][1:]
    return " ".join(text)

for video in tqdm(sorted(annotations.keys())):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    num_frames = len(annotations[video].keys())
    batch_id = []
    batch_image = []
    if video not in hazard_name_by_id:
        hazard_name_by_id[video] = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)
        
        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] in video_track_id[video] + video_track_id_tree[video] :
                x1, y1, x2, y2 = annotations[video][frame]['challenge_object'][i]['bbox']
                batch_id.append(annotations[video][frame]['challenge_object'][i]['track_id'])
                batch_image.append( frame_image[max(0, int(1.1*y1-0.1*y2)):min(int(1.1*y2-0.1*y1), frame_image.shape[0]), max(0, int(1.1*x1-0.1*x2)):min(int(1.1*x2-0.1*x1), frame_image.shape[1])] )
                if annotations[video][frame]['challenge_object'][i]['track_id'] not in hazard_name_by_id[video]:
                    hazard_name_by_id[video][annotations[video][frame]['challenge_object'][i]['track_id']] = {}
                
        if ((len(batch_id) >= batchsize) or (frame == num_frames-1)) and len(batch_id) > 0:
            inputs = processor(batch_image, ["car view of"]*len(batch_image), return_tensors="pt").to(device, torch.float16)
            output = model.generate(**inputs, max_length=64)
            output_text = [clean_text(processor.decode(output[i], skip_special_tokens=True)) for i in range(len(batch_id))]

            for track_id, image, text in zip(batch_id, batch_image, output_text):
                if text not in hazard_name_by_id[video][track_id]:
                    hazard_name_by_id[video][track_id][text] = 0.0
                hazard_name_by_id[video][track_id][text] += image.shape[0]*image.shape[1]
            
            batch_id = []
            batch_image = []

with open('hazard_name_by_id_blip2opt6_8b.pkl', 'wb') as handle:
    pickle.dump(hazard_name_by_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

In [ ]:
import cv2
import torch
import pickle
import numpy as np
import pandas as pd

from tqdm import *
from transformers import Blip2Processor, Blip2ForConditionalGeneration, BlipProcessor, BlipForConditionalGeneration, VisionEncoderDecoderModel, ViTImageProcessor, AutoTokenizer, InstructBlipProcessor, InstructBlipForConditionalGeneration
# from clip_interrogator import Config, Interrogator

batchsize = 16
df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
video_track_id = pickle.load(open("video_track_id.pkl", 'rb'))
video_track_id_tree = pickle.load(open("video_track_id_tree.pkl", 'rb'))

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# processor = BlipProcessor.from_pretrained("Salesforce/blip-image-captioning-large")
# model = BlipForConditionalGeneration.from_pretrained("Salesforce/blip-image-captioning-large", torch_dtype=torch.float16, device_map="cuda")

# processor = BlipProcessor.from_pretrained("noamrot/FuseCap")
# model= BlipForConditionalGeneration.from_pretrained("noamrot/FuseCap", torch_dtype=torch.float16, device_map="cuda")

processor = Blip2Processor.from_pretrained("Salesforce/blip2-opt-6.7b")
model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-opt-6.7b", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-flan-t5-xxl")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-flan-t5-xxl", load_in_4bit=True, torch_dtype=torch.float16, device_map="cuda")

# processor = AutoTokenizer.from_pretrained("nlpconnect/vit-gpt2-image-captioning")
# feature_extractor = ViTImageProcessor.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")
# model = VisionEncoderDecoderModel.from_pretrained("nlpconnect/vit-gpt2-image-captioning", torch_dtype=torch.float16, device_map="cuda")

# processor = Blip2Processor.from_pretrained("Salesforce/blip2-itm-vit-g")
# model = Blip2ForConditionalGeneration.from_pretrained("Salesforce/blip2-itm-vit-g", load_in_8bit=True, torch_dtype=torch.float16, device_map="cuda")

# model = InstructBlipForConditionalGeneration.from_pretrained("Salesforce/instructblip-vicuna-13b", torch_dtype=torch.float16, device_map="cuda")
# processor = InstructBlipProcessor.from_pretrained("Salesforce/instructblip-vicuna-13b", torch_dtype=torch.float16, device_map="cuda")

hazard_name_by_id = {} # pickle.load(open("hazard_name_by_id_final.pkl", "rb"))
def clean_text(text):
    text = text.replace("car view of ", "").replace(",", "").split()
    i = 1
    while i !=  len(text):
        if (text[i] == text[i-1]) or (text[i] == ""):
            text.pop(i)
        else:
            i += 1
    text[0] = text[0][0].upper() + text[0][1:]
    return " ".join(text)

for video in tqdm(sorted(annotations.keys())):
    video_stream = cv2.VideoCapture(f"COOOL/{video}.mp4")
    num_frames = len(annotations[video].keys())
    batch_image = []
    if video not in hazard_name_by_id:
        hazard_name_by_id[video] = {}
    for frame in sorted(annotations[video].keys()):
        ret, frame_image = video_stream.read()
        frame_image = cv2.cvtColor(frame_image, cv2.COLOR_BGR2RGB)

        for i in range(len(annotations[video][frame]['challenge_object'])):
            if annotations[video][frame]['challenge_object'][i]['track_id'] in video_track_id[video] + video_track_id_tree[video] :
                batch_image.append( frame_image )
                break

        if ((len(batch_image) >= batchsize) or (frame == num_frames-1)) and len(batch_image) > 0:
            inputs = processor(batch_image, ["car view of"]*len(batch_image), return_tensors="pt").to(device, torch.float16)
            output = model.generate(**inputs, max_length=64)
            output_text = [clean_text(processor.decode(output[i], skip_special_tokens=True)) for i in range(len(batch_image))]

            for image, text in zip(batch_image, output_text):
                if text not in hazard_name_by_id[video]:
                    hazard_name_by_id[video][text] = 0.0
                hazard_name_by_id[video][text] += image.shape[0]*image.shape[1]
            
            batch_image = []

with open('hazard_name_by_frame_blip2opt_4b.pkl', 'wb') as handle:
    pickle.dump(hazard_name_by_id, handle, protocol=pickle.HIGHEST_PROTOCOL)

# Post

In [ ]:
import nltk
import heapq
import string
import pickle
import numpy as np
import pandas as pd

from tqdm.notebook import *
from collections import Counter

df_final = pd.DataFrame()
annotations = pickle.load(open("annotations_public.pkl", 'rb'))
task1 = pickle.load(open("video_Driver_State_Changed.pkl", "rb"))
task2_1 = pickle.load(open("video_track_id_1.pkl", "rb"))
task2_2 = pickle.load(open("video_track_id_2.pkl", "rb"))
task3s = [pickle.load(open("hazard_name_by_id_blip2flan.pkl", "rb")),
          pickle.load(open("hazard_name_by_id_blip2opt6_4b.pkl", "rb")),
          pickle.load(open("hazard_name_by_id_blip.pkl", "rb")),
          pickle.load(open("hazard_name_by_id_blip2flan_4b.pkl", "rb")),
          pickle.load(open("hazard_name_by_id_vit.pkl", "rb")),
          pickle.load(open("hazard_name_by_id_blip2opt6_8b.pkl", "rb")),
         ]
task3_frame = pickle.load(open("hazard_name_by_frame_blip2opt_4b.pkl", "rb"))
task3 = {}
w = np.array([0.09252, 1.26830, 0.01384, 0.03213, 0.00016, 0.00574])

def mix(d, frame):
    new_d = {}
    for track_id, text_score in d.items():
        new_d[track_id] = d[track_id].copy()
        for other_track_id, text_score in d.items():
            for text, score in text_score.items():
                if text not in new_d[track_id]:
                    new_d[track_id][text] = 0.0
                new_d[track_id][text] += score/32.1098
        for text, score in frame.items():
            if text not in new_d[track_id]:
                new_d[track_id][text] = 0.0
            new_d[track_id][text] += score/(10*1920*1080)
    return new_d

for i in range(len(task3s)):
    for video in sorted(list(annotations.keys())):
        if video not in task3:
            task3[video] = {}
        for track_id, name_score in task3s[i][video].items():
            if track_id not in task3[video]:
                task3[video][track_id] = {}
            for name, score in name_score.items():
                if name not in task3[video][track_id]:
                    task3[video][track_id][name] = 0.0
                task3[video][track_id][name] += w[i]*score

for video in tqdm(sorted(list(annotations.keys()))):
    task3[video] = mix(task3[video], task3_frame[video])

remove = ["a", "the", "street", "walking", "on", "and", "with", "in", "of", "blurry", "road", "crossing", "background", "sitting", "foreground", "photo", "image", "running", "line", "down", "highway", "up", "front", "rain", "across", "driving", "at", "daytime", "night", "standing", "air", "through", "pickup", "day", "has", "roof", "driveway", "ford", "explorer", "her", "covered", "snow", "snowy", "water", "small", "sky", "over", "flying", "ha", "posing", "poses", 
          "cross", "is", "ground", "parking", "parked", "s", "out", "from", "by", "it", "other", "riding", "laptop", "computer", "keyboard", "television", "window", "lamp", "its", "his", "new", "picture", "city", "dmax", "bathroom", "king", "moon", "ufo", "suspect", "shirt", "object", "st", "johns", "logo", "thomas", "edward", "hitting", "mirror", "doing", "hazard", "dashcam", "shows", "this", "that", "middle", "presence", "which", "no", "haz", "there", "lot", "large", 
          "car", "drivers", "toyota", "yaris", "sidewalk", "mazda", 
         ]

reduce = ["white", "black", "yellow", "dark", "gray", "brown", "green", "accident", "two"] # new

def clean_text(s):
    s = s.lower()
    s = "".join([c for c in s if c in string.ascii_lowercase+" "])
    return s

def is_noun(word):
    word_tokenized = nltk.word_tokenize(word)
    pos = nltk.pos_tag(word_tokenized)[0][1]
    return pos in ['NN', 'NNS', 'NNP', 'NNPS']

def is_animal(word):
    synsets = nltk.corpus.wordnet.synsets(word)
    for synset in synsets:
        if ('animal' in synset.lexname()) or ('mammal' in synset.lexname()) or ('person' in synset.lexname()) or ('person' in word):
            return True
    return False

def clean_all_words(d):
    for track_id, text_score in d.items():
        clean_text_score = {}
        for text, score in text_score.items():
            for w in clean_text(text).split():
                if w not in clean_text_score:
                    clean_text_score[w] = 0.0
                clean_text_score[w] += score
        for w in remove:
            if w in clean_text_score:
                clean_text_score.pop(w)

        for w in reduce:
            if w in clean_text_score:
                clean_text_score[w] = clean_text_score[w]/3
                
        for w in clean_text_score:
            if is_noun(w):
                clean_text_score[w] = 2*clean_text_score[w]
            if is_animal(w):
                clean_text_score[w] = 1.5*clean_text_score[w]
        d[track_id] = {k: v for k, v in sorted(clean_text_score.items(), key=lambda item: item[1], reverse = True)}
    return d

def clean35(s, max_word = 20):
    if len(s) == 0:
        return -1
    c = 0
    i = 0
    r = []
    while (c <= 100) and (i<min(len(s), max_word+1)):
        r.append(s[i])
        c += len(s[i]) + 1
        i += 1
    return " ".join(r[:-1])

ids = []
Driver_State_Changed = []
Hazard_Track_all = [[] for i in range(23)]
Hazard_Track_name_all = [[] for i in range(23)]

for video in tqdm(sorted(list(annotations.keys()))):
    num_frames = len(annotations[video])
    Hazard_Track_video = [[-1]*num_frames for i in range(23)]
    Hazard_Track_name_video = [[-1]*num_frames for i in range(23)]
    ids += [f"{video}_{frame}" for frame in range(num_frames)]
    hazard_appear = False
    task3[video] = clean_all_words(task3[video])
    for frame in sorted(annotations[video].keys()):
        c = 0
        for track_id in Counter(task2_1[video] + task2_2[video]).keys():
            if track_id in [x['track_id'] for x in annotations[video][frame]['challenge_object']]:
                hazard_appear = True
                Hazard_Track_video[c][frame] = track_id
                if track_id in task3[video]:
                    Hazard_Track_name_video[c][frame] = clean35(sorted(task3[video][track_id], key=task3[video][track_id].get, reverse=True)[:36])
                else:
                    Hazard_Track_name_video[c][frame] = " "
                c += 1
        if (frame >= task1[video][1]):
            if not hazard_appear and task1[video][0] == 'random':
                Driver_State_Changed.append(False)
            else:
                Driver_State_Changed.append(True)
        else:
            Driver_State_Changed.append(False)
            
    for i in range(23):
        Hazard_Track_all[i] += Hazard_Track_video[i]
        Hazard_Track_name_all[i] += Hazard_Track_name_video[i]

df_final["ID"] = ids
df_final["Driver_State_Changed"] = Driver_State_Changed

for i in range(23):
    df_final[f"Hazard_Track_{i}"] = -1
    df_final[f"Hazard_Name_{i}"] = -1
    
for i in range(23):
    df_final[f"Hazard_Track_{i}"] = Hazard_Track_all[i]
    df_final[f"Hazard_Name_{i}"] = Hazard_Track_name_all[i]

df_final.to_csv("final.csv", index=False)